## Algorithm for Quantum Pattern Matching

## References:
### (1) Quantum Pattern Matching - P. Mateus and Y. Omar
### (2) Quantum Algorithms for pattern-matching in genomic sequences - Aritra Sarkar

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import(
  execute,
  Aer)
from math import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def randStr(A,N):
    bias = 1/A
    rbs = ""
    for i in range(0,N):
        rn = np.random.random()
        for j in range(0,A):
            if rn < (j+1)*bias:
                rbs = rbs + str(j)
                break
    return rbs

In [3]:
AS = {'0','1'}           # alphabet
A = len(AS)
N = 11
w = randStr(A,N)         # string (source)
M = 3
ans = 2
p = w[ans:ans+M]         # pattern (to be matched)

s = ceil(log2(N-M))+1
Q_D = s*M                # data qubits
Q_anc = 1                # ancilla qubits         
anc = Q_D
tot_q = Q_D + Q_anc      # total qubits

In [4]:
def nCX(q,circuit,nc,j,anc):
    nnc = len(nc)
    if nnc==1:
        circuit.h(q[j])
        circuit.cx(q[nc[0]],q[j])
        circuit.h(q[j])
    elif nnc==2:
        circuit.h(q[j])
        circuit.ccx(q[nc[0]],q[nc[1]],q[j])
        circuit.h(q[j])
    else:
        nch = ceil(nnc/2)
        c1 = nc[:nch]
        c2 = nc[nch:]
        c2.append(anc)
        nCX(q,circuit,c1,anc,nch)
        nCX(q,circuit,c2,j,nch-2)
        nCX(q,circuit,c1,anc,nch)
        nCX(q,circuit,c2,j,nch-2)
    return

# Initialization circuit:

In [5]:
def circ1(q,circuit,N,M,anc):
    for i in range(0,s):
        circuit.h(q[i])
    for m in range(0,M-1):
        for i in range(0,s):
            circuit.cx(q[m*s+i],q[(m+1)*s+i])
        for i in range(0,s):
            ic = (m+1)*s - (i+1)
            circuit.x(q[ic])
            nc = []
            for j in range(ic,(m+1)*s):
                nc.append(j)
            for j in range((m+2)*s-1,s+ic-1,-1):
                nCX(q,circuit,nc,j,anc)
            circuit.x(q[ic])
    return

# Oracle creation:

In [6]:
def circ2(q,circuit,bf,ind,N,M,anc):
    for i in range(0,len(bf)):
        if bf[i] == '1':
            fis = format(i, '0'+str(s)+'b')
            
            for fisi in range(0,s):
                if fis[fisi] == '0':
                    circuit.x(q[ind*s+fisi])
                    
            circuit.h(q[(ind+1)*s-1])
            
            nc = []
            for j in range(ind*s,(ind+1)*s-1):
                nc.append(j)
            nCX(q,circuit,nc,(ind+1)*s-1,anc-1)
            
            circuit.h(q[(ind+1)*s-1])
            
            for fisi in range(0,s):
                if fis[fisi] == '1':
                    circuit.x(q[ind*s+fisi])
    return

# Grover Implementation and Amplitude Amplification:

In [7]:
def circ3(q,circuit,N,M):
    for i in range(0,s*M):
        circuit.h(q[i])
        circuit.x(q[i])
    circuit.h(q[s*M-1])
    nc = []
    for j in range(0,s*M-1):
        nc.append(j)
    nCX(q,circuit,nc,s*M-1,s*M-1)
    circuit.h(q[s*M-1])
    for i in range(s*M):
        circuit.x(q[i])
        circuit.h(q[i])
    return

# Measurement Circuit:

In [8]:
def circ4(q,c,circuit,N,M):
    for i in range(s):
        circuit.measure(q[i],c[i])
    return

In [9]:
def QPM(AS,w,p):
    
    A = len(AS)
    N = len(w)
    M = len(p)
    Q_D = s*M
    Q_anc = 1
    anc = Q_D
    tot_q = Q_D + Q_anc
    
    q = QuantumRegister(tot_q,'phi')
    c = ClassicalRegister(tot_q)
    circuit = QuantumCircuit(q,c)
    
    circ1(q,circuit,N,M,anc)
    
    bfa = ''.join('1' if w[i] == '0' else '0' for i in range(N))
    bfc = ''.join('1' if w[i] == '1' else '0' for i in range(N))
    bfg = ''.join('1' if w[i] == '2' else '0' for i in range(N))
    bft = ''.join('1' if w[i] == '3' else '0' for i in range(N))
    
    for r in range(0,int(sqrt(N-M+1))):
        ind = np.random.randint(0,M-1)
        if p[ind] == '0':
            circ2(q,circuit,bfa,ind,N,M,anc)
        elif p[ind] == '1':
            circ2(q,circuit,bfc,ind,N,M,anc)
#         elif p[ind] == '2':
#             c2 = circ2(q,bfg,ind,N,M,anc)
#         elif p[ind] == '3':
#             c2 = circ2(q,bft,ind,N,M,anc)
                        
        circ3(q,circuit,N,M)
    
    circ4(q,c,circuit,N,M)
    
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(circuit, simulator, shots=1024)
    result = job.result()
    counts = result.get_counts(circuit)
    for i in counts:
        print(i,counts[i])

In [10]:
print("Alphabet: ",AS)
print("N: ",N,"M: ",M)
print("w: ",w)
print("p: ",p)
QPM(AS,w,p)

Alphabet:  {'0', '1'}
N:  11 M:  3
w:  10110001111
p:  110
0000000000001 59
0000000000101 64
0000000000011 55
0000000000010 68
0000000001010 69
0000000000100 71
0000000001100 65
0000000001110 53
0000000001000 64
0000000000000 65
0000000000111 58
0000000001111 67
0000000000110 66
0000000001101 69
0000000001001 65
0000000001011 66
